# Initialize things 

In [1]:
#import things

import cv2
import numpy  as np
from collections import deque
import time


In [2]:
from keras.models import load_model
cnn_model = load_model('emnist_cnn_model.h5')

In [3]:
prediction_dict = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l',
           12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w',
           23: 'x', 24: 'y', 25: 'z', 26: ''}

In [4]:
# define color limits

red_upper = (170,100,60)
red_lower = (180,255,255)

points = deque(maxlen = 512)
kernel = np.ones((5, 5), np.uint8)

#define blackboard and alphabets
blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
alphabet = np.zeros((300, 300, 3), dtype=np.uint8)

prediction = 26 #default prediction
center = None


# Capture the written letter

In [5]:

cap = cv2.VideoCapture(0)
    
while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    #hsv
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #red
    red = cv2.inRange(hsv, red_upper, red_lower)
    #Preprocessing the  image
    red = cv2.morphologyEx(red, cv2.MORPH_OPEN, kernel) 
    red_median = cv2.medianBlur(red, 15)
    
   
    #find contours
    contours, _ = cv2.findContours(red_median, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        contour_list = sorted(contours, key = cv2.contourArea, reverse = True)
        contour = contour_list[0]
        #find centroid
        M = cv2.moments(contour) #M will be a dict
        try:
            centroid_x = int(M["m10"]/M["m00"])
            centroid_y = int(M["m01"]/M["m00"])
        except:
            pass
        
        centroid = (centroid_x, centroid_y)
        #append centroid in points
        points.appendleft(centroid)
        #draw circle
        ((x,y), r) = cv2.minEnclosingCircle(contour)
        cv2.circle(frame, (int(x),int(y)), int(r), (255,215,0), 2)
   
    elif (len(contours)==0):
        if(len(points) != 0): #means there is some letter which was written and now the object is taken away
            blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
            blur = cv2.medianBlur(blackboard_gray, 15)
            blur = cv2.GaussianBlur(blur, (5, 5), 0)
            thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            
            blackboard_cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
            if len(blackboard_cnts) >= 1:
                #sorting all the contours to get the largest one
                contour = sorted(blackboard_cnts, key = cv2.contourArea, reverse = True)[0]

                if cv2.contourArea(contour) > 1000: #applying area condition to prevent noise from getting selected
                    x, y, w, h = cv2.boundingRect(contour)
                    alphabet = blackboard_gray[y-10:y + h + 10, x-10:x + w + 10]
                
                    try:
                        final_alphabet = cv2.resize(alphabet, (28,28))
                        # this is the final alphabet image 
                    except cv2.error as e:
                        points = deque(maxlen=512)
                        blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
                        continue
                        
                    final_alphabet = np.array(final_alphabet)
                    final_alphabet = final_alphabet.astype("float32")/255
                    
                    prediction = cnn_model.predict(final_alphabet.reshape(1,28,28,1))[0]
                    prediction = np.argmax(prediction)
                    print(f"The predicted alphabet was {prediction_dict[int(prediction)]}")#final output from model
                
        # Empty the points deque and the blackboard
        cv2.imshow("Blackboard", blackboard)
        blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
        points = deque(maxlen=512)
        
    #connecting the points with the lines
    for i in range(1, len(points)):
        if points[i - 1] is None or points[i] is None:
            continue
        cv2.line(frame, tuple(map(int,points[i-1])),tuple(map(int,points[i])) ,(0,0,0), 2)
        cv2.line(blackboard, tuple(map(int,points[i-1])), tuple(map(int,points[i])), (255,255,255), 8)
        
    # Show the frame
    

    cv2.putText(frame, "Convolution Neural Network:  " + str(prediction_dict[int(prediction)]), (10, 440), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.imshow("alphabets Recognition Real Time", frame) 
    
    if cv2.waitKey(1) &  0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()


The predicted alphabet was p
The predicted alphabet was w


# Displaying movie suggestions

In [6]:
import pyttsx3
import pandas as pd
import random

engine = pyttsx3.init()
titles = pd.read_csv('titles.csv')

#extracting content from the file
initial = prediction_dict[int(prediction)]
list_of_initial = titles[titles["title"].str.startswith(initial.upper())]
samples = list_of_initial.sample(n = 3).sort_values('year')
names = [samples["title"][int(index)] for index in samples.index]
years = [samples["year"][int(index)]for index in samples.index]
content = [names[i] +"   "+ str(years[i])  for i in range(len(names))]

#randomizing image to be shown
i  =random.randrange(1,6)
img_name = f"image_{i}.jpg"
image = cv2.imread(img_name,cv2.IMREAD_UNCHANGED)
r = 555.0 / image.shape[0]
dim = (int(image.shape[1] * r), 555)

# perform the resizing
image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
image= cv2.drawMarker(image,(190,75),(44,15,66),20,3,8)

while True:
    cv2.putText(image, "WE RECOMMEND YOU:", (200,80), cv2.FONT_HERSHEY_COMPLEX, 0.8, (84,14,150), 2)
    for i in range(len(names)):
        cv2.putText(image, content[i], (200,130+50*i), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (146,10,89), 2)
    cv2.imshow("Recommendations", image)
    if cv2.waitKey(1) & 0xFF == ord("s"):
        engine.setProperty('rate', 175)     # setting up new voice rate
        volume = engine.getProperty('volume')   #getting to know current volume level (min=0 and max=1)
        engine.setProperty('volume',2) 
        engine.say("\n".join(names))
        engine.runAndWait()
        
        break
    elif cv2.waitKey(1) & 0xFF == ord("q"):
        break
    


cv2.destroyAllWindows()
